<a href="https://colab.research.google.com/github/swapc740/TestAIAgent/blob/main/StockAIAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install ta
# Install Required Libraries
!pip install yfinance ta gradio pandas numpy scikit-learn

# Install Required Libraries (Run this separately in Colab or Terminal)
# !pip install yfinance gradio pandas numpy scikit-learn

import yfinance as yf
import pandas as pd
import gradio as gr
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Compute indicators manually without "ta" library
def compute_indicators(data):
    data["SMA_50"] = data["Close"].rolling(window=50).mean()
    data["SMA_200"] = data["Close"].rolling(window=200).mean()
    gain = data["Close"].diff()
    loss = -gain.clip(upper=0)
    gain = gain.clip(lower=0)
    avg_gain = gain.rolling(14, min_periods=1).mean()
    avg_loss = loss.rolling(14, min_periods=1).mean()
    rs = avg_gain / (avg_loss + 1e-6)  # Avoid division by zero
    data["RSI"] = 100 - (100 / (1 + rs))

    # Additional Indicators
    data["MACD"] = data["Close"].ewm(span=12, adjust=False).mean() - data["Close"].ewm(span=26, adjust=False).mean()
    data["Bollinger_Upper"] = data["Close"].rolling(window=20).mean() + (data["Close"].rolling(window=20).std() * 2)
    data["Bollinger_Lower"] = data["Close"].rolling(window=20).mean() - (data["Close"].rolling(window=20).std() * 2)

    data.dropna(inplace=True)
    return data

# Fetch stock data & compute indicators
def fetch_and_prepare_data(stock):
    if not stock.endswith((".NS", ".BO")):
        stock += ".NS"  # Default to NSE if no suffix provided

    data = yf.download(stock, period="5y")
    data = compute_indicators(data)
    return data

# Train Machine Learning Model
def train_model(data):
    data = data.copy()
    data["Target"] = (data["Close"].shift(-30) > data["Close"]).astype(int)  # Predict 30 days ahead
    data.dropna(inplace=True)

    X = data[["SMA_50", "SMA_200", "RSI", "MACD", "Bollinger_Upper", "Bollinger_Lower"]]
    y = data["Target"]

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    return model, scaler

# Train Model Initially
def initialize_model(stock):
    sample_data = fetch_and_prepare_data(stock)
    model, scaler = train_model(sample_data)
    return model, scaler

# Prediction Function
def predict(stock):
    model, scaler = initialize_model(stock)
    data = fetch_and_prepare_data(stock)
    latest_data = data[["SMA_50", "SMA_200", "RSI", "MACD", "Bollinger_Upper", "Bollinger_Lower"]].dropna().tail(1)

    if latest_data.empty:
        return "Not enough data to predict"

    latest_scaled = scaler.transform(latest_data)
    prediction = model.predict(latest_scaled)
    return "✅ Investment Recommended!" if prediction[0] == 1 else "❌ Investment Not Recommended!"

# Web Interface (Shareable Link)
def main():
    gr.Interface(
        fn=predict,
        inputs="text",
        outputs="text",
        title="Stock Investment Predictor",
        description="Enter a stock ticker symbol (e.g., RELIANCE.NS for NSE, TCS.BO for BSE) to get an investment recommendation."
    ).launch(share=True)

if __name__ == "__main__":
    main()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a89a476e5e36953687.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
